In [1]:
import numpy
import newspaper
import pandas as pd
from newspaper import Article
from bs4 import BeautifulSoup
import urllib.request
from IPython.display import HTML
import re
import requests
import datetime

In [2]:
urls = []
query = 'corona' # Use any keyword you would like
for i in range(1,21):
    urls.append(f"https://www.ndtv.com/page/topic-load-more?type=news&page={i}&query={query}")

### HTML Links

In [38]:
links = []
for i in range(len(urls)):
    html = requests.get(urls[i])
    soup = BeautifulSoup(html.text, 'lxml')
    for a in soup.find_all('a', href=True):
        links.append(a['href'])

### HTML Dates

In [42]:
len(links)

300

In [8]:
dates = []
for i in range(len(urls)):
    html = requests.get(urls[i])
    soup = BeautifulSoup(html.text, 'lxml')
    lists = soup.find_all('li')
    for i in lists:
        date = i.find(class_='list_dateline' ).text.split("|")[-1].strip()
        date1 = datetime.datetime.strptime(date,"%A %B %d, %Y")
        dates.append(date1)

In [40]:
i = len(dates)

In [41]:
links = links[0:i]

### Title

In [12]:
titles = []
for i in range(len(links)):
    html = requests.get(links[i])
    soup = BeautifulSoup(html.text, 'lxml')
    title = soup.title.get_text()
    titles.append(title)

### Newspaper

In [16]:
keyword = []
summ = []
for i in range(len(links)):
    article = Article(links[i])
    article.download()
    article.parse()
    article.nlp()
    keyword.append(article.keywords)
    summ.append(article.summary)

In [17]:
len(summ)

300

In [18]:
len(keyword)

300

In [ ]:
dat = {'keyword':keyword,'summary':summ}

In [ ]:
keys = pd.DataFrame(dat)

In [ ]:
keys.to_csv('keywordandsumm.csv')

### Final

In [19]:
link = pd.DataFrame(links, columns =['link'])
url_lockdown = link['link'].unique()

In [ ]:
url_lockdown = pd.DataFrame(url_lockdown,columns=['links'])

In [ ]:
url_lockdown = url_lockdown.iloc[0:885]
links = url_lockdown['links']

In [43]:
query = []
for i in range(len(links)):
    query.append("corona")

In [48]:
data = {'title': titles,'url':links,'date':dates,'query':query,'keyword':keyword,'summary':summ}

In [49]:
lock = pd.DataFrame(data)

In [50]:
lock.head()

,title,url,date,query,keyword,summary
0,Brand New Mahindra Ambulances Pressed In Coron...,https://www.carandbike.com/news/brand-new-mahi...,2020-06-15,corona,"[vehicle, mumbai, buy, brand, planning, ends, ...","In the present Covid-19 situation, are you pla..."
1,Coronavirus: Amit Shah's Surprise Delhi LNJP H...,https://www.ndtv.com/india-news/coronavirus-am...,2020-06-15,corona,"[shahs, allparty, covid, amit, delhi, surprise...",Delhi Coronavirus Cases: Amit Shah held an all...
2,"\r\n\tNDTV: Latest News, India News, Breaking ...",https://www.ndtv.com/india-news,2020-06-15,corona,"[business, songs, india, photos, breaking, bol...","Listen to the latest songs , only on JioSaavn.com"
3,Amit Shah Calls All-Party Meeting On Coronavir...,https://www.ndtv.com/india-news/home-minister-...,2020-06-14,corona,"[crisis, national, allparty, pradesh, shah, to...",Delhi Corona Cases: Amit Shah and Arvind Kejri...
4,"\r\n\tNDTV: Latest News, India News, Breaking ...",https://www.ndtv.com/india-news,2020-06-12,corona,"[business, songs, india, photos, breaking, bol...","Listen to the latest songs , only on JioSaavn.com"


In [63]:
lock1 = lock.drop_duplicates(subset = 'url')

In [65]:
lockf = lock1.reset_index(drop=True)

In [66]:
lockf.to_csv('corona_ndtv.csv')